The human heart, on average, will beat approximately 100,000 times each day and circulate roughly 2,000 gallons of blood through an extensive network of vessels stretching over 60,000 miles within the body.

Heart attack symptoms can manifest differently between genders, often being subtler in women. A female heart attack sufferer might experience a range of symptoms including a peculiar chest discomfort or pain, which could also spread to the arms, back, neck, jaw, or stomach, accompanied by difficulty breathing, nausea, and other varied signs. Men typically present more recognizable heart attack indicators such as pronounced chest pain or discomfort and stress, and similar to women, they might also suffer pain in several parts of the upper body along with symptoms like sweating and a sensation akin to indigestion.

For an organ that's merely the size of a large fist and tips the scales at just 8 to 12 ounces, the heart's workload is indeed formidable.

Orignal Data Source : [https://archive.ics.uci.edu/ml/datasets/Heart+Disease]

** The dataset was created by: **
1. Hungarian Institute of Cardiology. Budapest: Andras Janosi, M.D.
2. University Hospital, Zurich, Switzerland: William Steinbrunn, M.D.
3. University Hospital, Basel, Switzerland: Matthias Pfisterer, M.D.
4. V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.

**The dataset contains the following features:**

- **age**: The individual's age in years.
- **sex**: The individual's sex (1 = male, 0 = female).
- **cp**: Type of chest pain experienced.
  - 0: Asymptomatic
  - 1: Atypical angina
  - 2: Non-anginal pain
  - 3: Typical angina
- **trestbps**: The individual's resting blood pressure (in mm Hg on admission to the hospital).
- **chol**: The individual's cholesterol measurement in mg/dl.
- **fbs**: The individual's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false).
- **restecg**: Resting electrocardiographic results.
  - 0: Probable or definite left ventricular hypertrophy by Estes' criteria.
  - 1: Normal.
  - 2: Having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV).
- **thalach**: The individual's maximum heart rate achieved.
- **exang**: Exercise-induced angina (1 = yes; 0 = no).
- **oldpeak**: ST depression induced by exercise relative to rest.
- **slope**: The slope of the peak exercise ST segment.
  - 0: Downsloping.
  - 1: Flat.
  - 2: Upsloping.
- **ca**: The number of major vessels (0-3) colored by fluoroscopy.
- **thal**: The status of thalassemia, a blood disorder.
  - 0: NULL (dropped from the dataset).
  - 1: Fixed defect (no blood flow in some part of the heart).
  - 2: Normal blood flow.
  - 3: Reversible defect (blood flow is observed but it is not normal).
- **target**: Presence of heart disease (1 = no, 0 = yes).



In [1]:
#Installing the required packages
!pip install pgmpy

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination# Load the dataset
from pgmpy.estimators import ParameterEstimator, MaximumLikelihoodEstimator
from pgmpy.estimators import HillClimbSearch, BicScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import ParameterEstimator, MaximumLikelihoodEstimator
import pandas as pd

# Loading the Dataset
url = 'https://raw.githubusercontent.com/crbeveridge1065/ml_data/main/heart.csv'
df = pd.read_csv(url)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,class
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


I need to bin the continuous variables to make the network easier to work with....

In [2]:
# Age binning
age_bins = [0, 30, 60, 120]
age_labels = ['Young', 'Middle-aged', 'Elderly']
df['age_binned'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, include_lowest=True)

# Resting blood pressure binning
trestbps_bins = [0, 120, 140, 200]  # Assuming standard classification
trestbps_labels = ['Low', 'Normal', 'High']
df['trestbps_binned'] = pd.cut(df['trestbps'], bins=trestbps_bins, labels=trestbps_labels, include_lowest=True)

# Cholesterol binning
chol_bins = [0, 200, 239, 600]  # Assuming standard classification
chol_labels = ['Desirable', 'Borderline high', 'High']
df['chol_binned'] = pd.cut(df['chol'], bins=chol_bins, labels=chol_labels, include_lowest=True)

# Maximum heart rate achieved binning
# For 'thalach', we use quantiles as the values might be relative to age/fitness
thalach_bins = pd.qcut(df['thalach'], q=3, labels=['Below Average', 'Average', 'Above Average'])
df['thalach_binned'] = thalach_bins

# ST depression binning
oldpeak_bins = [0, 1, 2, 6]  # Custom bins based on clinical relevance
oldpeak_labels = ['Minimal', 'Moderate', 'Severe']
df['oldpeak_binned'] = pd.cut(df['oldpeak'], bins=oldpeak_bins, labels=oldpeak_labels, include_lowest=True)

# Drop the original continuous variables
df = df.drop(['age', 'trestbps', 'chol', 'thalach', 'oldpeak'], axis=1)




### Define the Structure

The structure of our Bayesian network is learned with HillClimbing search

In [3]:
hc = HillClimbSearch(df)
best_model = hc.estimate(scoring_method=BicScore(df))
edges = list(best_model.edges())
edges

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('sex', 'class'),
 ('sex', 'ca'),
 ('sex', 'age_binned'),
 ('cp', 'exang'),
 ('restecg', 'chol_binned'),
 ('exang', 'oldpeak_binned'),
 ('slope', 'thalach_binned'),
 ('thal', 'class'),
 ('thal', 'sex'),
 ('class', 'cp'),
 ('class', 'ca'),
 ('class', 'oldpeak_binned'),
 ('class', 'thalach_binned'),
 ('class', 'exang'),
 ('class', 'slope'),
 ('age_binned', 'trestbps_binned'),
 ('trestbps_binned', 'fbs'),
 ('thalach_binned', 'age_binned'),
 ('oldpeak_binned', 'slope'),
 ('oldpeak_binned', 'restecg')]

In [22]:
# use pgmpy to define the structure of the Bayesian network
model = BayesianNetwork(edges)




# Estimating the Conditional Probability Distributions

In this section of our analysis, I utilize the MaximumLikelihoodEstimator to learn the Conditional Probability Distributions (CPDs) of our Bayesian network model. This is a crucial step in the process of Bayesian modeling, as CPDs quantify the relationships between variables in the network. Here’s a breakdown of what this code does and why it's important:

The goal of MLE is to find the set of parameters (in our case, probability values for CPDs) that makes the observed data most likely. It does this by maximizing a likelihood function, which measures how well the data fits a given model.


In [23]:
# Use Maximum Likelihood Estimator to learn the CPDs based on the learned structure
model.fit(df, estimator=MaximumLikelihoodEstimator)

for cpd in model.get_cpds():
    print(cpd)

+--------+---------------------+---------+--------------------+---------------------+
| thal   | thal(0)             | thal(1) | thal(2)            | thal(3)             |
+--------+---------------------+---------+--------------------+---------------------+
| sex(0) | 0.42857142857142855 | 0.0625  | 0.4632352941176471 | 0.12926829268292683 |
+--------+---------------------+---------+--------------------+---------------------+
| sex(1) | 0.5714285714285714  | 0.9375  | 0.5367647058823529 | 0.8707317073170732  |
+--------+---------------------+---------+--------------------+---------------------+
+----------+---------+---------+-----+--------------------+---------------------+
| sex      | sex(0)  | sex(0)  | ... | sex(1)             | sex(1)              |
+----------+---------+---------+-----+--------------------+---------------------+
| thal     | thal(0) | thal(1) | ... | thal(2)            | thal(3)             |
+----------+---------+---------+-----+--------------------+-----------

# Performing Predictive Inference with Bayesian Networks

After constructing our Bayesian network and learning the conditional probability distributions (CPDs), the next crucial step is to utilize the model for predictive inference. Predictive inference allows us to estimate the probaility of certain outcomes, given a set of observed evidence. In this section, we demonstrate how to perform these inferences using hypothetical patient scenarios

## Variable Elimination Technique

To conduct predictive inference, we employ the 'VariableElimination' technique on our Bayesian model:

This method efficiently computes the probability distribution of a set of query variables, conditioned on observed evidence. It systematically eliminates variables that are not part of the query or evidence, simplifying computations and enabling us to focus on the variables of interest.

Alex: Middle-aged male with typical angina, normal cholesterol, and normal resting blood pressure.

In [6]:
infer = VariableElimination(model)

evidence_alex = {
    'age_binned': 'Middle-aged',
    'sex': 1,
    'cp': 3,
    'chol_binned': 'Desirable',  # Assuming good cholesterol level
    'trestbps_binned': 'Normal',  # Assuming normal blood pressure
    'thalach_binned': 'Above Average',  # Assuming good fitness
    'oldpeak_binned': 'Minimal'  # Assuming minimal ST depression
}

# Compute the probability of heart disease class given the evidence
result = infer.query(variables=['class'], evidence=evidence_alex)
print(result)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.1616 |
+----------+--------------+
| class(1) |       0.8384 |
+----------+--------------+


The results suggest that, given the specific health information provided for Alex, our Bayesian network model predicts a higher probability of heart disease presence (61.74%) than absence (38.26%). It's important to note that these probabilities are derived from the learned relationships and patterns within the data used to train the model and the specific evidence provided about Alex.

Dana: Middle-aged female with asymptomatic chest pain, borderline high cholesterol, and normal resting blood pressure.

In [7]:
evidence_dana = {
    'age_binned': 'Middle-aged',
    'sex': 0,
    'cp': 0,
    'chol_binned': 'Borderline high',
    'trestbps_binned': 'Normal',
    'thalach_binned': 'Average',
    'oldpeak_binned': 'Moderate'
}

# Compute the probability of heart disease class given the evidence
result = infer.query(variables=['class'], evidence=evidence_dana)
print(result)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.5469 |
+----------+--------------+
| class(1) |       0.4531 |
+----------+--------------+


This result suggests that Dana has a 60.10% chance of not having heart disease and a 39.90% chance of having heart disease. Compared to Alex, Dana's health profile indicates a lower risk of heart disease according to the model.

Eli: Elderly male with non-anginal pain, high cholesterol, but low resting blood pressure.

In [8]:
evidence_eli = evidence_eli = {
    'age_binned': 'Elderly',
    'sex': 1,
    'cp': 2,
    'chol_binned': 'High',
    'trestbps_binned': 'Low',
    'thalach_binned': 'Average',
    'oldpeak_binned': 'Severe'
}

# Compute the probability of heart disease class given the evidence
result = infer.query(variables=['class'], evidence=evidence_eli)

print(result)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.6082 |
+----------+--------------+
| class(1) |       0.3918 |
+----------+--------------+


These probabilities suggest that, according to the Bayesian network model and the evidence provided for Eli, there's a 59.72% chance that Eli does not have heart disease and a 40.28% chance that he does. Despite some high-risk factors like high cholesterol and severe ST depression, the model assesses Eli's overall risk of having heart disease as lower than the risk of not having it.

Casey: Young male with atypical angina, desirable cholesterol levels, and low resting blood pressure.

In [9]:
evidence_casey = {
    'age_binned': 'Young',
    'sex': 1,
    'cp': 1,
    'chol_binned': 'Desirable',
    'trestbps_binned': 'Low',
    'thalach_binned': 'Above Average',
    'oldpeak_binned': 'Minimal'
}

# Compute the probability of heart disease class given the evidence
result = infer.query(variables=['class'], evidence=evidence_casey)
print(result)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.0867 |
+----------+--------------+
| class(1) |       0.9133 |
+----------+--------------+


Casey's health profile—being young, male, with a certain type of chest pain, desirable cholesterol levels, low resting blood pressure, above-average maximum heart rate achieved, and minimal ST depression—leads the model to estimate a 63.47% chance of having heart disease and a 36.53% chance of not having it. This high probability might seem counterintuitive given the positive health indicators, suggesting that the model might be picking up on nuanced patterns or that Casey's profile matches patterns of individuals with heart disease within the data the model was trained on.

Blake: Elderly female with non-anginal pain, high cholesterol, and high resting blood pressure.

In [10]:
evidence_blake = {
    'age_binned': 'Elderly',
    'sex': 0,
    'cp': 2,
    'chol_binned': 'High',
    'trestbps_binned': 'High',
    'thalach_binned': 'Below Average',
    'oldpeak_binned': 'Severe'
}

# Compute the probability of heart disease class given the evidence
result = infer.query(variables=['class'], evidence=evidence_blake)
print(result)


+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.7416 |
+----------+--------------+
| class(1) |       0.2584 |
+----------+--------------+


Blake's results show a more balanced probability distribution between having and not having heart disease, indicating that her risk factors contribute to a moderate likelihood of heart disease according to the model.

In [11]:
from pgmpy.sampling import BayesianModelSampling

# Create a sampler object
sampler = BayesianModelSampling(model)

# Generate a random sample of 10,000 records
samples = sampler.forward_sample(size=10000)

# Filter samples where exercise-induced angina ('exang') is present
samples_with_exang = samples[samples['exang'] == 1]

# Now, print the probability distribution of chest pain type ('cp') given the filtered samples
cp_distribution_given_exang = samples_with_exang['cp'].value_counts(normalize=True)
cp_distribution_given_exang


  0%|          | 0/14 [00:00<?, ?it/s]

0    0.833185
2    0.097604
3    0.036084
1    0.033126
Name: cp, dtype: float64

This distribution offers valuable insights into the correlation between exercise-induced angina and the types of chest pain these patients experience. Notably, a significant majority of patients with exercise-induced angina experience Type 0 chest pain, which is classified as asymptomatic. This suggests that among this specific group, the chest pain most commonly does not align with the traditional symptoms of angina that are easily recognizable.

Types 2 and 3 chest pain, which represent non-anginal and atypical angina pain, respectively, account for a smaller portion of the distribution, indicating that these types of chest pain are less common among patients with exercise-induced angina. Type 1, typical angina, is the least common, which may initially seem counterintuitive but underscores the complexity of diagnosing heart conditions based solely on symptom presentation.

# Probabilistic Inference Using Belief Propagation (BP):

In this part of our analysis, we employ the Belief Propagation (BP) algorithm to conduct probabilistic inference on our Bayesian network. This method allows us to calculate the posterior probabilities of certain events, given observed evidence, by propagating beliefs (probabilities) through the network in an efficient manner.

In [12]:
from pgmpy.inference import BeliefPropagation
bp = BeliefPropagation(model)
bp.calibrate()

# Query 1:alex
q1 = bp.query(variables=['class'], evidence=evidence_alex)
print(q1)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.1616 |
+----------+--------------+
| class(1) |       0.8384 |
+----------+--------------+


In [13]:
# Query 2: Dana
q2 = bp.query(variables=['class'], evidence=evidence_dana)
print(q2)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.5469 |
+----------+--------------+
| class(1) |       0.4531 |
+----------+--------------+


In [14]:

# Query 3: Eli
q3 = bp.query(variables=['class'], evidence=evidence_eli)

# Print the result
print(q3)


+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.6082 |
+----------+--------------+
| class(1) |       0.3918 |
+----------+--------------+


In [15]:
# Query 4: Casey
q4 = bp.query(variables=['class'], evidence=evidence_casey)

# Print the result
print(q4)

+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |          nan |
+----------+--------------+
| class(1) |          nan |
+----------+--------------+


/usr/local/lib/python3.10/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:488: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()


In [16]:
# Query 5: Blake
q5 = bp.query(variables=['class'], evidence=evidence_blake)

# Print the result
print(q5)


+----------+--------------+
| class    |   phi(class) |
+==========+==============+
| class(0) |       0.7416 |
+----------+--------------+
| class(1) |       0.2584 |
+----------+--------------+


# Try Your Own Heart Disease Risk Predictions

I've developed a predictive model that assesses the risk of heart disease based on various health indicators. To make this model accessible and interactive, we've created a user-friendly interface using Gradio. This tool allows you to input hypothetical patient scenarios and receive instant predictions on the likelihood of heart disease.

### How to Use the Interface:
* Input Data: Use the sliders and dropdown menus to enter health characteristics for a hypothetical patient. These characteristics include chest pain type, the presence of exercise-induced angina, resting electrocardiographic results, sex, thalassemia status, the slope of the peak exercise ST segment, and more.

* View Predictions: After entering the data, the model will compute the probability distribution of heart disease presence (class 0 for absence and class 1 for presence) based on the input evidence.

* Explore Different Scenarios: Feel free to adjust the inputs to explore how different health profiles affect the predicted risk of heart disease. This interactive exploration can provide insights into the factors that the model considers significant in assessing heart disease risk.


In [17]:
!pip install gradio==3.43.1
import gradio as gr
from pgmpy.inference import VariableElimination

# Define the prediction function with arguments for all variables
def predict_class_probability(cp, exang, restecg, sex, thal, slope, age_binned, trestbps_binned, chol_binned, thalach_binned, oldpeak_binned, ca):
    evidence = {
        'cp': cp,
        'exang': exang,
        'restecg': restecg,
        'sex': sex,
        'thal': thal,
        'slope': slope,
        'age_binned': age_binned,
        'trestbps_binned': trestbps_binned,
        'chol_binned': chol_binned,
        'thalach_binned': thalach_binned,
        'oldpeak_binned': oldpeak_binned,
        'ca': ca
    }

    # Create an inference object
    infer = VariableElimination(model)

    # Query the probability distribution of heart disease class given the evidence
    prob_class = infer.query(variables=['class'], evidence=evidence).values.flatten()

    # Normalize probabilities
    prob_class_normalized = prob_class / prob_class.sum()

    assert prob_class_normalized.sum() == 1, 'Probabilities do not sum to 1'


    class_probabilities = {f'Class {idx}': prob for idx, prob in enumerate(prob_class_normalized)}

    return class_probabilities

# Create the Gradio interface with sliders for numerical variables and dropdowns for categorical variables
interface = gr.Interface(fn=predict_class_probability,
                         inputs=[gr.inputs.Slider(minimum=0, maximum=3, step=1, label='Chest Pain Type (cp)'),
                                 gr.inputs.Slider(minimum=0, maximum=1, step=1, label='Exercise-induced Angina (exang)'),
                                 gr.inputs.Slider(minimum=0, maximum=2, step=1, label='Resting Electrocardiographic Results (restecg)'),
                                 gr.inputs.Slider(minimum=0, maximum=1, step=1, label='Sex (sex)'),
                                 gr.inputs.Slider(minimum=0, maximum=3, step=1, label='Thalassemia (thal)'),
                                 gr.inputs.Slider(minimum=0, maximum=2, step=1, label='Slope of Peak Exercise ST Segment (slope)'),
                                 gr.inputs.Radio(['Young', 'Middle-aged', 'Elderly'], label='Age Group (age_binned)'),
                                 gr.inputs.Radio(['Low', 'Normal', 'High'], label='Resting Blood Pressure Group (trestbps_binned)'),
                                 gr.inputs.Radio(['Desirable', 'Borderline high', 'High'], label='Cholesterol Group (chol_binned)'),
                                 gr.inputs.Radio(['Below Average', 'Average', 'Above Average'], label='Maximum Heart Rate Achieved Group (thalach_binned)'),
                                 gr.inputs.Radio(['Minimal', 'Moderate', 'Severe'], label='ST Depression Group (oldpeak_binned)'),
                                 gr.inputs.Slider(minimum=0, maximum=3, step=1, label='Number of Major Vessels Colored by Fluoroscopy (ca)')],
                         outputs=gr.outputs.Label(num_top_classes=2))

# Launch the interface
interface.launch(share=True)


<ipython-input-17-d3c00886a74e>:40: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Slider(minimum=0, maximum=3, step=1, label='Chest Pain Type (cp)'),
<ipython-input-17-d3c00886a74e>:40: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Slider(minimum=0, maximum=3, step=1, label='Chest Pain Type (cp)'),
<ipython-input-17-d3c00886a74e>:41: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Slider(minimum=0, maximum=1, step=1, label='Exercise-induced Angina (exang)'),
<ipython-input-17-d3c00886a74e>:41: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Slider(minimum=0, maximum=1, step=1, label='Exercise-induced Angina (exang)'),
<ipython-input-17-

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://43d5696de1fc9e61dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
